In [5]:
#Import Statements
import deepchem as dc
from rdkit import Chem
from rdkit.Chem import Draw
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import layers, losses

import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import (precision_recall_curve, average_precision_score, PrecisionRecallDisplay)
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_auc_score

2023-04-08 13:42:58.559395: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Tox21 Assay List: <br>
<ol>
    <li> NR-AR </li>
    <li> NR-AR-LBD </li>
    <li> NR-AhR </li>
    <li> NR-Aromatase </li>
    <li> NR-ER </li>
    <li> NR-ER-LBD </li>
    <li> NR-PPAR-gamma </li>
    <li> SR-ARE </li>
    <li> SR-ATAD5 </li>
    <li> SR-HSE </li>
    <li> SR-MMP </li>
    <li> SR-p53 </li>
</ol>

In [2]:
assays = ['NR-AR', 'NR-AR-LBD','NR-AhR', 'NR-Aromatase', 'NR-ER', 'NR-ER-LBD', 'NR-PPAR-gamma',
         'SR-ARE', 'SR-ATAD5', 'SR-HSE', 'SR-MMP', 'SR-p53']

## SmilesToImage Featuriser - CNN Model

In [6]:
tasks_smiles, datasets_smiles, transformers_smiles = dc.molnet.load_tox21(
    featurizer = dc.feat.SmilesToImage(img_size=80, img_spec='std'),
    save_dir=r'C:\Users\ym20201\Documents\Datasets',
    data_dir=r'C:\Users\ym20201\Documents\Datasets')

splitter = dc.splits.RandomSplitter()

train_data_smiles, valid_data_smiles, test_data_smiles = datasets_smiles

In [5]:
#SmilesToImage Model 7
smiles2img_model = dc.models.CNN(
    n_tasks = len(tasks_smiles), # Num of tasks, i.e. width of y
    n_features=len(train_data_smiles.X[2]), # number of features, i.e. width of X
    dims=1,
    layer_filter=[500,500,200],
    mode='classification',
    weight_init_stddevs=0.02, 
    bias_init_consts=1.0,
    dropouts=0.5,
    dense_layer_size=[500,200,100],
    activation_fns=['relu'],
    uncertainty=False,
    pool_type='max',
    residual=False,
    padding='valid') 

2023-04-07 17:10:57.278459: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
smiles2img_model.fit(
    train_data_smiles,
    nb_epoch=20)

0.8116668701171875

In [8]:
smiles2img_pred = smiles2img_model.predict(test_data_smiles)

In [ ]:
#Function for accessing y_true and y_pred of each assay
def y_true(assay_num, test_data):
    y_true = []
    for i in range(len(test_data.y)):
        y_true.append(test_data.y[i][assay_num - 1])
        
    return y_true

def y_pred(assay_num, pred_data):
    y_pred = []
    for i in range(len(pred_data)):
        y_pred.append( pred_data[i][assay_num - 1][0])
#         y_pred = pred_data[i][assay_num - 1] #not sure which one
    
    return y_pred

In [ ]:
#y_true of each assay
smiles2img_y_true_1 = y_true(1, test_data_smiles)
smiles2img_y_true_2 = y_true(2, test_data_smiles)
smiles2img_y_true_3 = y_true(3, test_data_smiles)
smiles2img_y_true_4 = y_true(4, test_data_smiles)
smiles2img_y_true_5 = y_true(5, test_data_smiles)
smiles2img_y_true_6 = y_true(6, test_data_smiles)
smiles2img_y_true_7 = y_true(7, test_data_smiles)
smiles2img_y_true_8 = y_true(8, test_data_smiles)
smiles2img_y_true_9 = y_true(9, test_data_smiles)
smiles2img_y_true_10 = y_true(10, test_data_smiles)
smiles2img_y_true_11 = y_true(11, test_data_smiles)
smiles2img_y_true_12 = y_true(12, test_data_smiles)

smiles2img_y_true = [smiles2img_y_true_1, smiles2img_y_true_2, smiles2img_y_true_3, smiles2img_y_true_4, smiles2img_y_true_5, 
        smiles2img_y_true_6, smiles2img_y_true_7, smiles2img_y_true_8, smiles2img_y_true_9, smiles2img_y_true_10,
        smiles2img_y_true_11, smiles2img_y_true_12]

In [ ]:
#y_pred of each assay
smiles2img_y_pred_1 = y_pred(1, smiles2img_pred)
smiles2img_y_pred_2 = y_pred(2, smiles2img_pred)
smiles2img_y_pred_3 = y_pred(3, smiles2img_pred)
smiles2img_y_pred_4 = y_pred(4, smiles2img_pred)
smiles2img_y_pred_5 = y_pred(5, smiles2img_pred)
smiles2img_y_pred_6 = y_pred(6, smiles2img_pred)
smiles2img_y_pred_7 = y_pred(7, smiles2img_pred)
smiles2img_y_pred_8 = y_pred(8, smiles2img_pred)
smiles2img_y_pred_9 = y_pred(9, smiles2img_pred)
smiles2img_y_pred_10 = y_pred(10, smiles2img_pred)
smiles2img_y_pred_11 = y_pred(11, smiles2img_pred)
smiles2img_y_pred_12 = y_pred(12, smiles2img_pred)

In [ ]:
def round_pred(y_pred):
    y_pred_new = []
    for i in range(len(y_pred)):
        if y_pred[i] < 0.5:
            new = 0
        else:
            new = 1
        y_pred_new.append(new)
    return y_pred_new

In [ ]:
#Rounding predicted probabilities to binary values
smiles2img_y_pred_new_1 = round_pred(smiles2img_y_pred_1)
smiles2img_y_pred_new_2 = round_pred(smiles2img_y_pred_2)
smiles2img_y_pred_new_3 = round_pred(smiles2img_y_pred_3)
smiles2img_y_pred_new_4 = round_pred(smiles2img_y_pred_4)
smiles2img_y_pred_new_5 = round_pred(smiles2img_y_pred_5)
smiles2img_y_pred_new_6 = round_pred(smiles2img_y_pred_6)
smiles2img_y_pred_new_7= round_pred(smiles2img_y_pred_7)
smiles2img_y_pred_new_8 = round_pred(smiles2img_y_pred_8)
smiles2img_y_pred_new_9 = round_pred(smiles2img_y_pred_9)
smiles2img_y_pred_new_10 = round_pred(smiles2img_y_pred_10)
smiles2img_y_pred_new_11 = round_pred(smiles2img_y_pred_11)
smiles2img_y_pred_new_12 = round_pred(smiles2img_y_pred_12)

In [ ]:
#Adding the values into arrays
smiles2img_y_pred = [smiles2img_y_pred_1, smiles2img_y_pred_2, smiles2img_y_pred_3, smiles2img_y_pred_4, smiles2img_y_pred_5, 
             smiles2img_y_pred_6, smiles2img_y_pred_7, smiles2img_y_pred_8, smiles2img_y_pred_9, smiles2img_y_pred_10,
            smiles2img_y_pred_11, smiles2img_y_pred_12]

smiles2img_y_pred_new = [smiles2img_y_pred_new_1, smiles2img_y_pred_new_2, smiles2img_y_pred_new_3, smiles2img_y_pred_new_4, smiles2img_y_pred_new_5, 
             smiles2img_y_pred_new_6, smiles2img_y_pred_new_7, smiles2img_y_pred_new_8, smiles2img_y_pred_new_9, smiles2img_y_pred_new_10,
            smiles2img_y_pred_new_11, smiles2img_y_pred_new_12]

## ECFP Featuriser - MultitaskClassifier Model

In [ ]:
#CircularFingerprint (ECFP) featuriser
tasks_ecfp, datasets_ecfp, transformers_ecfp = dc.molnet.load_tox21(
    featurizer=dc.feat.CircularFingerprint(),
    save_dir=r'C:\Users\ym20201\Documents\Datasets',
    data_dir=r'C:\Users\ym20201\Documents\Datasets')

splitter = dc.splits.RandomSplitter()

train_data_ecfp, valid_data_ecfp,test_data_ecfp = datasets_ecfp

In [ ]:
#ECFP Model 5
ecfp_model = dc.models.RobustMultitaskClassifier(
    n_tasks = len(tasks_ecfp),
    n_features = len(valid_data_ecfp.X[3]),
    layer_sizes=[500,500,200],
    weight_init_stddevs=0.02, 
    bias_init_consts=1.0,
    weight_decay_penalty=0.0,
    weight_decay_penalty_type='12',
    dropouts=[0.8,0.5,0.0],
    activation_fns=['relu'],  
    n_classes=12,
    learning_rate=0.01,
    batch_size=100)

In [ ]:
ecfp_model.fit(train_data_ecfp,
              nb_epoch=10)

In [ ]:
ecfp_pred = ecfp_model.predict(test_data_ecfp)

In [ ]:
#y_true of each assay
ecfp_y_true_1 = y_true(1, test_data_ecfp)
ecfp_y_true_2 = y_true(2, test_data_ecfp)
ecfp_y_true_3 = y_true(3, test_data_ecfp)
ecfp_y_true_4 = y_true(4, test_data_ecfp)
ecfp_y_true_5 = y_true(5, test_data_ecfp)
ecfp_y_true_6 = y_true(6, test_data_ecfp)
ecfp_y_true_7 = y_true(7, test_data_ecfp)
ecfp_y_true_8 = y_true(8, test_data_ecfp)
ecfp_y_true_9 = y_true(9, test_data_ecfp)
ecfp_y_true_10 = y_true(10, test_data_ecfp)
ecfp_y_true_11 = y_true(11, test_data_ecfp)
ecfp_y_true_12 = y_true(12, test_data_ecfp)

ecfp_y_true = [ecfp_y_true_1, ecfp_y_true_2, ecfp_y_true_3, ecfp_y_true_4, ecfp_y_true_5, 
        ecfp_y_true_6, ecfp_y_true_7, ecfp_y_true_8, ecfp_y_true_9, ecfp_y_true_10,
        ecfp_y_true_11, ecfp_y_true_12]

In [ ]:
#y_pred of each assay
ecfp_y_pred_1 = y_pred(1, ecfp_pred)
ecfp_y_pred_2 = y_pred(2, ecfp_pred)
ecfp_y_pred_3 = y_pred(3, ecfp_pred)
ecfp_y_pred_4 = y_pred(4, ecfp_pred)
ecfp_y_pred_5 = y_pred(5, ecfp_pred)
ecfp_y_pred_6 = y_pred(6, ecfp_pred)
ecfp_y_pred_7 = y_pred(7, ecfp_pred)
ecfp_y_pred_8 = y_pred(8, ecfp_pred)
ecfp_y_pred_9 = y_pred(9, ecfp_pred)
ecfp_y_pred_10 = y_pred(10, ecfp_pred)
ecfp_y_pred_11 = y_pred(11, ecfp_pred)
ecfp_y_pred_12 = y_pred(12, ecfp_pred)

In [ ]:
#Rounding predicted probabilities to binary values
ecfp_y_pred_new_1 = round_pred(ecfp_y_pred_1)
ecfp_y_pred_new_2 = round_pred(ecfp_y_pred_2)
ecfp_y_pred_new_3 = round_pred(ecfp_y_pred_3)
ecfp_y_pred_new_4 = round_pred(ecfp_y_pred_4)
ecfp_y_pred_new_5 = round_pred(ecfp_y_pred_5)
ecfp_y_pred_new_6 = round_pred(ecfp_y_pred_6)
ecfp_y_pred_new_7= round_pred(ecfp_y_pred_7)
ecfp_y_pred_new_8 = round_pred(ecfp_y_pred_8)
ecfp_y_pred_new_9 = round_pred(ecfp_y_pred_9)
ecfp_y_pred_new_10 = round_pred(ecfp_y_pred_10)
ecfp_y_pred_new_11 = round_pred(ecfp_y_pred_11)
ecfp_y_pred_new_12 = round_pred(ecfp_y_pred_12)

In [ ]:
#Adding the values into arrays
ecfp_y_pred = [ecfp_y_pred_1, ecfp_y_pred_2, ecfp_y_pred_3, ecfp_y_pred_4, ecfp_y_pred_5, 
             ecfp_y_pred_6, ecfp_y_pred_7, ecfp_y_pred_8, ecfp_y_pred_9, ecfp_y_pred_10,
            ecfp_y_pred_11, ecfp_y_pred_12]

ecfp_y_pred_new = [ecfp_y_pred_new_1, ecfp_y_pred_new_2, ecfp_y_pred_new_3, ecfp_y_pred_new_4, ecfp_y_pred_new_5, 
             ecfp_y_pred_new_6, ecfp_y_pred_new_7, ecfp_y_pred_new_8, ecfp_y_pred_new_9, ecfp_y_pred_new_10,
            ecfp_y_pred_new_11, ecfp_y_pred_new_12]

## ConvMol Featuriser - GraphConv Model

In [ ]:
#GraphConv featuriser
tasks_convmol, datasets_convmol, transformers_convmol = dc.molnet.load_tox21(
    featurizer = dc.feat.ConvMolFeaturizer(),
    save_dir=r'C:\Users\ym20201\Documents\Datasets',
    data_dir=r'C:\Users\ym20201\Documents\Datasets')

splitter = dc.splits.RandomSplitter()

train_data_convmol, valid_data_convmol,test_data_convmol = datasets_convmol

In [ ]:
#Model 6
convmol_model = dc.models.GraphConvModel(
    n_tasks = len(tasks_convmol),
    graph_conv_layers=[32,32,32],
    dense_layer_size=128,
    dropout=0.0,
    mode='classification',
    number_atom_features=75,#default value
    batch_normalize=True,
    uncertainty=False,
    n_classes=12,
    learning_rate=0.01,
    batch_size=100)

In [ ]:
convmol_model.fit(
    train_data_convmol,
    nb_epoch=10)

In [ ]:
convmol_pred = convmol_model.predict(test_data_convmol)

In [ ]:
#y_true of each assay
convmol_y_true_1 = y_true(1, test_data_convmol)
convmol_y_true_2 = y_true(2, test_data_convmol)
convmol_y_true_3 = y_true(3, test_data_convmol)
convmol_y_true_4 = y_true(4, test_data_convmol)
convmol_y_true_5 = y_true(5, test_data_convmol)
convmol_y_true_6 = y_true(6, test_data_convmol)
convmol_y_true_7 = y_true(7, test_data_convmol)
convmol_y_true_8 = y_true(8, test_data_convmol)
convmol_y_true_9 = y_true(9, test_data_convmol)
convmol_y_true_10 = y_true(10, test_data_convmol)
convmol_y_true_11 = y_true(11, test_data_convmol)
convmol_y_true_12 = y_true(12, test_data_convmol)

convmol_y_true = [convmol_y_true_1, convmol_y_true_2, convmol_y_true_3, convmol_y_true_4, convmol_y_true_5, 
        convmol_y_true_6, convmol_y_true_7, convmol_y_true_8, convmol_y_true_9, convmol_y_true_10,
        convmol_y_true_11, convmol_y_true_12]

In [ ]:
#y_pred of each assay
convmol_y_pred_1 = y_pred(1, convmol_pred)
convmol_y_pred_2 = y_pred(2, convmol_pred)
convmol_y_pred_3 = y_pred(3, convmol_pred)
convmol_y_pred_4 = y_pred(4, convmol_pred)
convmol_y_pred_5 = y_pred(5, convmol_pred)
convmol_y_pred_6 = y_pred(6, convmol_pred)
convmol_y_pred_7 = y_pred(7, convmol_pred)
convmol_y_pred_8 = y_pred(8, convmol_pred)
convmol_y_pred_9 = y_pred(9, convmol_pred)
convmol_y_pred_10 = y_pred(10, convmol_pred)
convmol_y_pred_11 = y_pred(11, convmol_pred)
convmol_y_pred_12 = y_pred(12, convmol_pred)

In [ ]:
#Rounding predicted probabilities to binary values
convmol_y_pred_new_1 = round_pred(convmol_y_pred_1)
convmol_y_pred_new_2 = round_pred(convmol_y_pred_2)
convmol_y_pred_new_3 = round_pred(convmol_y_pred_3)
convmol_y_pred_new_4 = round_pred(convmol_y_pred_4)
convmol_y_pred_new_5 = round_pred(convmol_y_pred_5)
convmol_y_pred_new_6 = round_pred(convmol_y_pred_6)
convmol_y_pred_new_7= round_pred(convmol_y_pred_7)
convmol_y_pred_new_8 = round_pred(convmol_y_pred_8)
convmol_y_pred_new_9 = round_pred(convmol_y_pred_9)
convmol_y_pred_new_10 = round_pred(convmol_y_pred_10)
convmol_y_pred_new_11 = round_pred(convmol_y_pred_11)
convmol_y_pred_new_12 = round_pred(convmol_y_pred_12)

In [ ]:
#Adding the values into arrays
convmol_y_pred = [convmol_y_pred_1, convmol_y_pred_2, convmol_y_pred_3, convmol_y_pred_4, convmol_y_pred_5, 
             convmol_y_pred_6, convmol_y_pred_7, convmol_y_pred_8, convmol_y_pred_9, convmol_y_pred_10,
            convmol_y_pred_11, convmol_y_pred_12]

convmol_y_pred_new = [convmol_y_pred_new_1, convmol_y_pred_new_2, convmol_y_pred_new_3, convmol_y_pred_new_4, convmol_y_pred_new_5, 
             convmol_y_pred_new_6, convmol_y_pred_new_7, convmol_y_pred_new_8, convmol_y_pred_new_9, convmol_y_pred_new_10,
            convmol_y_pred_new_11, convmol_y_pred_new_12]

## Balanced Accuracy Score

In [ ]:
def balanced_accuracy(y_true, y_pred):
    balanced_acc = []
    for i in range(len(y_true)):
        b_acc = balanced_accuracy_score(y_true[i], y_pred[i])
        balanced_acc.append(b_acc)
    return balanced_acc

In [ ]:
smiles2img_balanced_acc = balanced_accuracy(smiles2img_y_true, smiles2img_y_pred_new)
ecfp_balanced_acc = balanced_accuracy(ecfp_y_true, ecfp_y_pred_new)
convmol_balanced_acc = balanced_accuracy(convmol_y_true, convmol_y_pred_new)

In [ ]:
means= [np.mean(smiles2img_balanced_acc), np.mean(ecfp_balanced_acc), np.mean(convmol_balanced_acc)]
means

In [ ]:
medians= [np.median(smiles2img_balanced_acc), np.median(ecfp_balanced_acc), np.median(convmol_balanced_acc)]
medians

In [ ]:
#Grouped bar plot
fig,ax = plt.subplots(figsize=(22,10))

width = 0.2

x=np.arange(12)
plt.bar(x-0.2, smiles2img_balanced_acc, width, label='SmilesToImage', color='steelblue')
plt.bar(x, ecfp_balanced_acc, width, label='ECFP', color='gold')
plt.bar(x+0.2, convmol_balanced_acc, width, label='ConvMol', color='indianred')
plt.title('Balanced Accuracy Score of Different Featurisers', fontsize=16)
plt.ylabel('Balanced Accuracy Score', fontsize=14)
plt.xlabel('Tox21 Assays', fontsize=14)

#Bar labels
plt.bar_label(ax.containers[0], fmt='%.3f', fontsize=12)
plt.bar_label(ax.containers[1], fmt='%.3f', fontsize=12)
plt.bar_label(ax.containers[2], fmt='%.3f', fontsize=12)

#Assay labels
plt.xticks(x, assays, fontsize=12)
plt.yticks(fontsize=12)
plt.legend()
# plt.savefig('Overall BA.png')

In [ ]:
#Grouped bar plot - NR
fig,ax = plt.subplots(figsize=(22,10))

width = 0.2

x=np.arange(7)

plt.bar(x-0.2, smiles2img_balanced_acc[0:7], width, label='SmilesToImage', color='steelblue')
plt.bar(x, ecfp_balanced_acc[0:7], width, label='ECFP', color='gold')
plt.bar(x+0.2, convmol_balanced_acc[0:7], width, label='ConvMol', color='indianred')

plt.title('Balanced Accuracy Score of Different Featurisers for Nuclear Receptor (NR) Panel', fontsize=16)
plt.ylabel('Balanced Accuracy Score', fontsize=14)
plt.xlabel('Nuclear Receptor Panel', fontsize=14)

#Bar labels
plt.bar_label(ax.containers[0], fmt='%.3f', fontsize=12)
plt.bar_label(ax.containers[1], fmt='%.3f', fontsize=12)
plt.bar_label(ax.containers[2], fmt='%.3f', fontsize=12)

#Assay labels
plt.xticks(x, assays[0:7],fontsize=12)
plt.yticks(fontsize=12)
plt.legend()
# plt.savefig('Overall BA-NR.png')

In [ ]:
#Grouped bar plot - SR
fig,ax = plt.subplots(figsize=(20,10))

width = 0.2

x=np.arange(5)

plt.bar(x-0.2, smiles2img_balanced_acc[7:12], width, label='SmilesToImage', color='steelblue')
plt.bar(x, ecfp_balanced_acc[7:12], width, label='ECFP', color='gold')
plt.bar(x+0.2, convmol_balanced_acc[7:12], width, label='ConvMol', color='indianred')

plt.title('Balanced Accuracy Score of Different Featurisers for Stress Response (SR) Panel', fontsize=16)
plt.ylabel('Balanced Accuracy Score', fontsize=14)
plt.xlabel('Stress Response Panel', fontsize=14)

#Bar labels
plt.bar_label(ax.containers[0], fmt='%.3f', fontsize=12)
plt.bar_label(ax.containers[1], fmt='%.3f', fontsize=12)
plt.bar_label(ax.containers[2], fmt='%.3f', fontsize=12)

#Assay labels
plt.xticks(x, assays[7:12],fontsize=12)
plt.yticks(fontsize=12)
plt.legend()
# plt.savefig('Overall BA-SR.png')

From the evaluation of the three models, the CNN model with the SmilesToImage featuriser was observed to have the highest average balanced accuracy score. This model was used to build an autoencoder to improve model performance. 

Additionally, to investigate the effect of epochs on average balanced accuracy score, the CNN model was run with different epochs, ranging from 1-15. The results were compiled in a separate Excel spreadsheet for evaluation.

## Building an autoencoder using the CNN model

In [7]:
#Splitting SmilesToImage featurised dataset into smaller train,valid,test datasets
splitter = dc.splits.RandomSplitter()

train_smiles_small, valid_smiles_small,test_smiles_small = splitter.train_valid_test_split(
    datasets_smiles[0], 
    frac_train = 0.8, frac_valid = 0.08, frac_test = 0.12)

In [ ]:
#Filtering out empty arrays in featurised datasets
train_feat = []
for i in range(len(train_smiles_small.X)):
    if train_smiles_small.X[i].shape != (0,):
        train_feat.append(train_smiles_small.X[i])
        
valid_feat = []
for i in range(len(valid_smiles_small.X)):
    if valid_smiles_small.X[i].shape != (0,):
        valid_feat.append(valid_smiles_small.X[i])

test_feat = []
for i in range(len(test_smiles_small.X)):
    if test_smiles_small.X[i].shape != (0,):
        test_feat.append(test_smiles_small.X[i])

In [ ]:
#Converting numpy array to tensor
train_tensor = []
for i in range(len(train_feat)):
    tensor = tf.convert_to_tensor(train_feat[i])
    train_tensor.append(tensor)
    
valid_tensor = []
for i in range(len(valid_feat)):
    tensor_v = tf.convert_to_tensor(valid_feat[i])
    valid_tensor.append(tensor_v)
    
test_tensor = []
for i in range(len(test_feat)):
    tensor_t = tf.convert_to_tensor(test_feat[i])
    test_tensor.append(tensor_t)

To start building the autoencoder, the keras blog was used for reference of building a 2D convolutional autoencoder. Firstly, a simple convolutional autoencoder of one convolutional layer was built. The autoencoder was improved by adding more layers such as the max pooling layer and flatten layer.

In [ ]:
# #Simple convolutional autoencoder

# # This is the size of our encoded representations
# encoding_dim = 32  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# input_img = layers.Input(shape=(40,40,1))

# #2D-Convolutional layer
# Conv2d_1 = layers.Conv2D(filters=32,
#                           kernel_size=3,
#                           activation='relu',
#                           dilation_rate=2)(input_img)

# # "encoded" is the encoded representation of the input
# encoded = layers.Dense(encoding_dim, activation='relu')(Conv2d_1)

# # "decoded" is the loss reconstruction of the input
# decoded = layers.Dense(784, activation='relu')(encoded)

# # This model maps an input to its reconstruction
# autoencoder = keras.Model(input_img, decoded)

# # This model maps an input to its encoded representation
# encoder = keras.Model(input_img, encoded)

# # This is our encoded (32-dimensional) input
# encoded_input = keras.Input(shape=(encoding_dim,
# # Retrieve the last layer of the autoencoder model
# decoder_layer = autoencoder.layers[-1]

# # Create the decoder model
# decoder = keras.Model(encoded_input, decoder_layer(encoded_input))

In [ ]:
# #Adding more layers to the convolutional autoencoder

# # This is the size of our encoded representations
# encoding_dim = 32  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# input_img = layers.Input(shape=(40,40,1))

# #2D-Convolutional layer 1
# Conv2D_1 = layers.Conv2D(filters=32,
#                           kernel_size=3,
#                           activation='relu',
#                           dilation_rate=2)(input_img)

# #Max pooling layer 1
# MaxPool2D_1 = layers.MaxPooling2D((2, 2), padding='same')(Conv2D_1)

# #2D-Convolutional layer 2
# Conv2D_2 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(MaxPool2D_1)

# #Max pooling layer 2
# MaxPool2D_2 = layers.MaxPooling2D((2, 2), padding='same')(Conv2D_2)

# #Flatten layer
# Flatten = layers.Flatten()(MaxPool2D_2)

# encoded = layers.Dense(encoding_dim, activation='relu')(Flatten)


# # "decoded" is the loss reconstruction of the input
# decoded = layers.Dense(784, activation='relu')(encoded)

# # This model maps an input to its reconstruction
# autoencoder = keras.Model(input_img, decoded)

# # This model maps an input to its encoded representation
# encoder = keras.Model(input_img, encoded)

# # This is our encoded (32-dimensional) input
# encoded_input = keras.Input(shape=(encoding_dim,))

# # Retrieve the last layer of the autoencoder model
# decoder_layer = autoencoder.layers[-1]

# # Create the decoder model
# decoder = keras.Model(encoded_input, decoder_layer(encoded_input))

In [ ]:
# #Compiling autoencoder
# autoencoder.compile(optimizer='adam', loss='mean_squared_error')
# autoencoder.summary()

In [ ]:
# #Training the autoencoder
# autoencoder_test = autoencoder.fit(x=train_tensor,
#                                    y=train_tensor,
#                                    epochs=10,
#                                    batch_size=512,
#                                    shuffle=True,
#                                   validation_data=(valid_tensor, valid_tensor))

The above autoencoder model was not able to train on the training data so another method of implementing the keras model was used. 

In [3]:
# #Constructing the model

# keras_model = Sequential()
# keras_model.add(layers.Conv2D(filters=32,
#                           kernel_size=3,
#                           activation='relu',
#                           dilation_rate=2,
#                             input_shape=(80, 80, 1)))
# keras_model.add(layers.MaxPooling2D(pool_size=(2,2)))
# keras_model.add(layers.Dense(50, activation='relu'))
# keras_model.add(layers.Dense(1))

# keras_model.build()

In [ ]:
# keras_model.compile(optimizer='adam', loss='mean_squared_error')
# keras_model.summary()

In [ ]:
# keras_model.fit(x=train_tensor,
#                 y=train_tensor,
#                 epochs=10,
#                 batch_size=512,
#                 shuffle=True,
#                validation_data=(valid_tensor, valid_tensor))

However, this autoencoder also does not work. This could be due to the Tox21 dataset containing multi-dimensional data. To improve on the training of the model, the Tox21 dataset could be split into each of the 12 assays and train the model with each individual assay to start with.